# LLM-архиватор. Эксперименты hyper-prior для steering vector

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.notebook import tqdm
import time
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.cluster import KMeans

In [ ]:
def clear_memory(var_names=None):
    if var_names is None:
        var_names = [
            'model','tokenizer','enc','dec','logits','probs','inp','ctx',
            'ids','recovered_ids','all_bits','all_original_ids'
        ]
    for var in var_names:
        if var in globals():
            del globals()[var]
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

## Общие и вспомогательные функции кодирования и декодирования

In [ ]:
def probs_to_cdf_int(prob, total=1 << 20):
    cdf = np.cumsum(prob, dtype=np.float64)
    cdf_int = np.floor(cdf * total).astype(np.int64)
    cdf_int = np.maximum.accumulate(cdf_int)
    cdf_int = np.concatenate(([0], cdf_int))
    cdf_int[-1] = total
    return cdf_int

In [ ]:
class ArithmeticEncoder:
    def __init__(self, precision=32):
        self.precision = precision
        self.half  = 1 << (precision - 1)
        self.quarter = self.half >> 1
        self.mask  = (1 << precision) - 1
        self.low   = 0
        self.high  = self.mask
        self.pending = 0
        self.out = []

    def update(self, cdf_low, cdf_high, total):
        rng = self.high - self.low + 1
        self.high = self.low + (rng * cdf_high) // total - 1
        self.low  = self.low + (rng * cdf_low)  // total

        # renormalize
        while True:
            # E1: MSB equal
            if self.high < self.half:
                self._emit(0)
            elif self.low >= self.half:
                self._emit(1)
                self.low  -= self.half
                self.high -= self.half
            # E3: underflow
            elif self.low >= self.quarter and self.high < 3 * self.quarter:
                self.pending += 1
                self.low  -= self.quarter
                self.high -= self.quarter
            else:
                break

            # shift out
            self.low   = (self.low   << 1) & self.mask
            self.high  = ((self.high << 1) & self.mask) | 1

    def _emit(self, bit):
        self.out.append(bit)
        for _ in range(self.pending):
            self.out.append(1 - bit)
        self.pending = 0

    def finish(self):
        self.pending += 1
        if self.low < self.quarter:
            self._emit(0)
        else:
            self._emit(1)
        for _ in range(self.precision):
            self.out.append((self.low >> (self.precision - 1)) & 1)
            self.low = (self.low << 1) & self.mask

In [ ]:
class ArithmeticDecoder:
    def __init__(self, bits, precision=32):
        self.bits = bits
        self.precision = precision
        self.half  = 1 << (precision - 1)
        self.quarter = self.half >> 1
        self.mask  = (1 << precision) - 1

        self.low   = 0
        self.high  = self.mask
        self.value = 0
        self.idx   = 0
        for _ in range(precision):
            self.value = ((self.value << 1) & self.mask) | self._read()

    def _read(self):
        if self.idx < len(self.bits):
            b = self.bits[self.idx]
            self.idx += 1
            return b
        return 0

    def decode(self, cdf_int, total):
        # find symbol
        rng = self.high - self.low + 1
        scaled = ((self.value - self.low + 1) * total - 1) // rng
        symbol = np.searchsorted(cdf_int, scaled, side='right') - 1

        # narrow interval
        c_lo, c_hi = cdf_int[symbol], cdf_int[symbol+1]
        self.high = self.low + (rng * c_hi) // total - 1
        self.low  = self.low + (rng * c_lo) // total

        # renormalize
        while True:
            if self.high < self.half:
                pass
            elif self.low >= self.half:
                self.value -= self.half
                self.low   -= self.half
                self.high  -= self.half
            elif self.low >= self.quarter and self.high < 3 * self.quarter:
                self.value -= self.quarter
                self.low   -= self.quarter
                self.high  -= self.quarter
            else:
                break

            self.low   = (self.low   << 1) & self.mask
            self.high  = ((self.high << 1) & self.mask) | 1
            self.value = ((self.value << 1) & self.mask) | self._read()

        return symbol

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def load_model(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    model.eval()
    model.to(device)

    return tokenizer, model

In [ ]:
def get_data(size: int, tokenizer, path: str = 'enwik8') -> list[int]:
    with open(path, 'rb') as f:
        data = f.read(size)
    text = data.decode('latin-1')
    return data

In [ ]:
class TextChunkDataset(Dataset):
    def __init__(self, path, chunk_size, tokenizer, max_bytes=None):
        raw = open(path, 'rb').read()
        if max_bytes:
            raw = raw[:max_bytes]
        self.chunks = []
        for i in range(0, len(raw), chunk_size):
            text = raw[i:i+chunk_size].decode('latin-1', errors='ignore')
            ids  = tokenizer.encode(text, add_special_tokens=False)
            if len(ids) > 1:
                self.chunks.append(ids)

    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        return torch.tensor(self.chunks[idx], dtype=torch.long)

In [ ]:
def create_data_loader(
    tokenizer,
    chunk_size: int,
    max_bytes: int
):
    dataset = TextChunkDataset('enwik8', chunk_size, tokenizer, max_bytes=max_bytes)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    return dataset, loader

## Обучение soft-prompt

In [ ]:
def create_soft_prompt(n_soft: int, model):
    H = model.config.hidden_size
    soft_prompt = nn.Parameter(torch.randn(n_soft, H, device=device))
    return soft_prompt

In [ ]:
def train_soft_prompt(
    loader,
    model,
    soft_prompt: torch.nn.Parameter,
    optimizer,
    epochs: int
):
    n_soft = soft_prompt.size(0)

    for epoch in range(1, epochs + 1):
        total_loss = 0.0
        for batch in tqdm(loader, desc=f"Epoch {epoch}"):
            ids = batch[0].to(device)
            L = ids.size(0)
            if L < 2:
                continue

            optimizer.zero_grad()

            embeds = model.get_input_embeddings()(ids.unsqueeze(0))
            sp     = soft_prompt.unsqueeze(0)
            inp_e  = torch.cat([sp, embeds], dim=1)
            attn   = torch.ones(inp_e.shape[:2], device=device)

            labels = torch.full((1, n_soft + L), -100, dtype=torch.long, device=device)
            labels[0, n_soft:] = ids

            out  = model(inputs_embeds=inp_e, attention_mask=attn, labels=labels)
            loss = out.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg = total_loss / len(loader)
        print(f"Epoch {epoch} done — avg loss: {avg:.6f}")

## Кодирование и декодирование с soft-prompt

In [ ]:
def encode_with_soft_prompt(
    dataset,
    model,
    soft_prompt: torch.nn.Parameter,
    total: int = 1 << 30
):
    chunk_data = []
    start_time = time.perf_counter()
    max_pe = model.config.max_position_embeddings
    n_soft = soft_prompt.shape[0]

    for ids in tqdm(dataset, desc="Encoding chunks"):
        ids = ids.to(device)
        enc = ArithmeticEncoder()
        sp = soft_prompt.unsqueeze(0)

        for i in tqdm(range(1, ids.size(0)), desc='  Tokens in chunk', leave=False):
            hist = ids[max(0, i - (max_pe - n_soft)):i].unsqueeze(0)
            embeds = model.get_input_embeddings()(hist)
            inp_emb = torch.cat([sp, embeds], dim=1)
            attn_mask = torch.ones(inp_emb.shape[:2], device=device)

            with torch.no_grad():
                out = model(inputs_embeds=inp_emb, attention_mask=attn_mask)
            logits = out.logits[0, -1]
            probs = torch.softmax(logits, dim=-1).cpu().numpy()

            cdf = probs_to_cdf_int(probs, total)
            token_id = ids[i].item()
            enc.update(cdf[token_id], cdf[token_id + 1], total)

        enc.finish()
        chunk_data.append({
            'ids': ids.cpu(),
            'bits': enc.out
        })

    encoding_time = time.perf_counter() - start_time
    print(f"Encoding done in {encoding_time:.2f}s")
    return chunk_data

In [ ]:
def decode_with_soft_prompt(
    chunk_data,
    model,
    soft_prompt: torch.nn.Parameter,
    total: int = 1 << 30
):
    start_time = time.perf_counter()
    all_rec = []
    max_pe = model.config.max_position_embeddings
    n_soft = soft_prompt.shape[0]
    sp = soft_prompt.unsqueeze(0)

    for entry in tqdm(chunk_data, desc="Decoding chunks"):
        ids  = entry['ids']
        bits = entry['bits']
        dec  = ArithmeticDecoder(bits)

        rec = [ids[0].item() if hasattr(ids[0], 'item') else ids[0]]
        for i in tqdm(range(1, len(ids)), desc='  Tokens in chunk', leave=False):
            hist = rec[max(0, i - (max_pe - n_soft)):i]
            input_ids = torch.tensor(hist, dtype=torch.long, device=device).unsqueeze(0)
            embeds = model.get_input_embeddings()(input_ids)
            inp_emb = torch.cat([sp, embeds], dim=1)
            attn_mask = torch.ones(inp_emb.shape[:2], device=device)

            with torch.no_grad():
                out = model(inputs_embeds=inp_emb, attention_mask=attn_mask)
            logits = out.logits[0, -1]
            probs  = torch.softmax(logits, dim=-1).cpu().numpy()
            cdf    = probs_to_cdf_int(probs, total)
            rec.append(dec.decode(cdf, total))

        all_rec.extend(rec)

    decoding_time = time.perf_counter() - start_time
    print(f"Decoding done in {decoding_time:.2f}s")
    return all_rec

## Hyper prior

### Гауссов гипер‑приор

In [ ]:
def gaussian_hyper_prior(
    sigma: float = 1.0,
    c_min: float = -5.0,
    c_max: float = 5.0,
    n_bins: int = 256,
    total_prior: int = 1 << 30
):
    delta = (c_max - c_min) / n_bins
    # Центры бинов
    bin_centers = np.linspace(c_min + delta/2, c_max - delta/2, n_bins)

    # Вычисляем PMF гауссиана на центрах бинов
    prior_pmf = np.exp(-0.5 * (bin_centers / sigma)**2) / (sigma * np.sqrt(2 * np.pi))
    prior_pmf /= prior_pmf.sum()

    # Интегрируем PMF в целочисленный CDF
    cdf = np.floor(np.cumsum(prior_pmf) * total_prior).astype(np.int64)
    prior_cdf_int = np.concatenate(([0], np.maximum.accumulate(cdf)))
    prior_cdf_int[-1] = total_prior

    return prior_cdf_int, delta, c_min, prior_pmf

In [ ]:
def quantize_c(c_tensor, c_min):
    arr = c_tensor.detach().cpu().numpy().flatten()
    idx = np.clip(((arr - c_min)/delta).round().astype(int), 0, 255)
    return idx

def dequantize_c(idx, shape, c_min):
    centers = (np.arange(256) + 0.5) * delta + c_min
    return torch.tensor(centers[idx].reshape(shape), dtype=torch.float32, device=device)

In [ ]:
def encode_soft_prompt(soft_prompt, prior_cdf_int, c_min, total_prior = 1 << 30):
    # Квантизуем значения soft_prompt в индексы бинов
    c_idx = quantize_c(soft_prompt, c_min)

    enc = ArithmeticEncoder()
    for s in c_idx:
        lo = prior_cdf_int[s]
        hi = prior_cdf_int[s + 1]
        enc.update(lo, hi, total_prior)
    enc.finish()

    return enc.out

In [ ]:
def decode_soft_prompt(
    c_bits: bytes,
    prior_cdf_int: np.ndarray,
    shape: tuple[int, ...],
    c_min,
    total_prior: int = 1 << 30
):
    dec = ArithmeticDecoder(c_bits)
    n_elem = int(np.prod(shape))
    rec_idx = [dec.decode(prior_cdf_int, total_prior) for _ in range(n_elem)]
    rec_idx = np.array(rec_idx, dtype=np.int64)
    soft_prompt_rec = dequantize_c(rec_idx, shape, c_min)
    return soft_prompt_rec

In [ ]:
def compare_sequences(orig, dec):
    for k, (o, d) in enumerate(zip(orig, dec)):
        if o != d:
            print(f"❌  Расхождение на позиции {k}: orig={o}  decoded={d}")
            return
    if len(orig) != len(dec):
        print(f"❌  Длины списков отличаются: orig={len(orig)}  decoded={len(dec)}")
        return
    print("✅  Совпадают полностью!")

In [ ]:
def compression_stats(data: bytes, enc_out: bytes):
    original_bits = len(data) * 8
    compressed_bits = len(enc_out)
    ratio = compressed_bits / original_bits

    print(f"Исходный размер:   {original_bits} бит")
    print(f"Размер после сжатия: {compressed_bits} бит")
    print(f"Коэффициент сжатия: {ratio:.4f}")

In [ ]:
def evaluate_steering_coding(
    soft_prompt: torch.Tensor,
    soft_prompt_rec: torch.Tensor,
    prior_pmf: np.ndarray,
    c_bits: bytes
):
    num_dims = soft_prompt.numel()
    bits_used = len(c_bits)
    bits_per_dim = bits_used / num_dims

    pmf_pos = prior_pmf[prior_pmf > 0]
    H_coord = -np.sum(pmf_pos * np.log2(pmf_pos))
    H_total = H_coord * num_dims

    rec = torch.as_tensor(soft_prompt_rec,
                         device=soft_prompt.device,
                         dtype=soft_prompt.dtype)
    mse = torch.mean((soft_prompt - rec) ** 2).item()

    original_bits = num_dims * 32
    comp_ratio = bits_used / original_bits

    print(f"Число координат steering‑вектора: {num_dims}")
    print(f"Битов использовано для кодирования: {bits_used}")
    print(f"Битов на координату: {bits_per_dim:.4f}")
    print(f"Теоретическая энтропия на координату: {H_coord:.4f} бит, всего ≈{H_total:.2f} бит")
    print(f"MSE: {mse:.6e}")
    print(f"Коэффициент компрессии steering‑вектора: {comp_ratio:.4f}")

In [ ]:
def decode_text_from_ids(ids, tokenizer) -> str:
    return tokenizer.decode(
        ids,
        clean_up_tokenization_spaces=False,
        skip_special_tokens=False
    )

In [ ]:
tokenizer, model = load_model('EleutherAI/pythia-70m')

In [ ]:
soft_prompt = create_soft_prompt(200, model)

In [ ]:
data = get_data(50000, tokenizer)

In [ ]:
dataset, loader = create_data_loader(tokenizer, chunk_size=2000, max_bytes=50000)

In [ ]:
model.train()
optimizer = torch.optim.Adam([soft_prompt], lr=5e-4)

In [ ]:
train_soft_prompt(loader, model, soft_prompt, optimizer, 500)

Epoch 1:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1 done — avg loss: 3.693652


Epoch 2:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 2 done — avg loss: 3.675581


Epoch 3:   0%|          | 0/25 [00:00<?, ?it/s]

...

Epoch 498 done — avg loss: 2.914423


Epoch 499:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 499 done — avg loss: 2.913058


Epoch 500:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 500 done — avg loss: 2.911371


In [ ]:
prior_cdf_int, delta, c_min, prior_pmf = gaussian_hyper_prior(sigma=1.0, c_min=-5.0, c_max=5.0, n_bins=256)

In [ ]:
c_bits = encode_soft_prompt(soft_prompt, prior_cdf_int, c_min)

In [ ]:
soft_prompt_rec = decode_soft_prompt(c_bits, prior_cdf_int, soft_prompt.shape, c_min)

In [ ]:
model.eval();

In [ ]:
chunk_data = encode_with_soft_prompt(
    dataset, model, soft_prompt_rec)

Encoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Encoding done in 200.25s


In [ ]:
all_rec = decode_with_soft_prompt(
    chunk_data, model, soft_prompt_rec)

Decoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Decoding done in 203.10s


In [ ]:
orig_ids = sum((entry['ids'].tolist() for entry in chunk_data), [])
encoded_output = sum([entry['bits'] for entry in chunk_data], [])

In [ ]:
compare_sequences(orig_ids, all_rec)

✅  Совпадают полностью!


In [ ]:
compression_stats(data, encoded_output)

Исходный размер:   400000 бит
Размер после сжатия: 54190 бит
Коэффициент сжатия: 0.1355


In [ ]:
evaluate_steering_coding(soft_prompt, soft_prompt_rec, prior_pmf, c_bits)

Число координат steering‑вектора: 102400
Битов использовано для кодирования: 690257
Битов на координату: 6.7408
Теоретическая энтропия на координату: 6.7252 бит, всего ≈688655.97 бит
MSE: 5.098574e-04
Коэффициент компрессии steering‑вектора: 0.2106


### Эмпирический гипер‑приор

In [ ]:
def empirical_hyper_prior(soft_prompt, bins: int = 256, total_prior: int = 1 << 30,
                          lo_pct: float = 1.0, hi_pct: float = 99.0):
    # Выгружаем значения в numpy и распрямляем
    vals = soft_prompt.detach().cpu().numpy().flatten()

    # Берём 1–99-й перцентиль
    vmin, vmax = np.percentile(vals, [lo_pct, hi_pct])

    # Считаем гистограмму и нормируем в PMF
    hist, _ = np.histogram(vals, bins=bins, range=(vmin, vmax))
    prior_pmf = hist.astype(float)
    prior_pmf /= prior_pmf.sum()

    # Интегрируем PMF в CDF и масштабируем
    cdf = np.floor(np.cumsum(prior_pmf) * total_prior).astype(np.int64)
    # Добавляем 0 в начало и обеспечиваем неубывающую функцию
    prior_cdf_int = np.concatenate(([0], np.maximum.accumulate(cdf)))
    # Гарантируем, что последний элемент равен total_prior
    prior_cdf_int[-1] = total_prior

    # Ширина одного бина
    delta = (vmax - vmin) / bins

    return prior_cdf_int, delta, vmin, prior_pmf

In [ ]:
prior_cdf_int, delta, c_min, prior_pmf = empirical_hyper_prior(soft_prompt)

In [ ]:
c_bits = encode_soft_prompt(soft_prompt, prior_cdf_int, c_min)

In [ ]:
soft_prompt_rec = decode_soft_prompt(c_bits, prior_cdf_int, soft_prompt.shape, c_min)

In [ ]:
evaluate_steering_coding(soft_prompt, soft_prompt_rec, prior_pmf, c_bits)

Число координат steering‑вектора: 102400
Битов использовано для кодирования: 795086
Битов на координату: 7.7645
Теоретическая энтропия на координату: 7.6999 бит, всего ≈788474.58 бит
MSE: 4.301889e-03
Коэффициент компрессии steering‑вектора: 0.2426


In [ ]:
chunk_data = encode_with_soft_prompt(
    dataset, model, soft_prompt_rec)

Encoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Encoding done in 198.17s


In [ ]:
all_rec = decode_with_soft_prompt(
    chunk_data, model, soft_prompt_rec)

Decoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Decoding done in 204.26s


In [ ]:
orig_ids = sum((entry['ids'].tolist() for entry in chunk_data), [])
encoded_output = sum([entry['bits'] for entry in chunk_data], [])

In [ ]:
compare_sequences(orig_ids, all_rec)

✅  Совпадают полностью!


In [ ]:
compression_stats(data, encoded_output)

Исходный размер:   400000 бит
Размер после сжатия: 54687 бит
Коэффициент сжатия: 0.1367


### Lloyd–Max (1D k‑means) гипер‑приор

In [ ]:
def lloyd_max_hyper_prior(soft_prompt,
                          bins: int = 256,
                          total_prior: int = 1 << 30,
                          sample_size: int = 100_000,
                          random_state: int = 0):
    vals = soft_prompt.detach().cpu().numpy().flatten()
    N    = len(vals)

    # Берём подвыборку для кластеризации
    if N > sample_size:
        idx_sample = np.random.choice(N, sample_size, replace=False)
        sample = vals[idx_sample].reshape(-1, 1)
    else:
        sample = vals.reshape(-1, 1)

    # KMeans → Lloyd‑Max центроиды
    kmeans = KMeans(n_clusters=bins, random_state=random_state).fit(sample)
    centroids = np.sort(kmeans.cluster_centers_.flatten())

    # Квантуем все значения в ближайший центроид
    diff = np.abs(vals[:, None] - centroids[None, :])
    idx  = np.argmin(diff, axis=1)

    # Собираем гистограмму попаданий → PMF
    hist = np.bincount(idx, minlength=bins).astype(float)
    prior_pmf = hist / hist.sum()

    # CDF → целочисленный для арифметического кода
    cdf = np.floor(np.cumsum(prior_pmf) * total_prior).astype(np.int64)
    prior_cdf_int = np.concatenate(([0], np.maximum.accumulate(cdf)))
    prior_cdf_int[-1] = total_prior

    return prior_cdf_int, centroids, prior_pmf

In [ ]:
def quantize_c_lm(c_tensor, centroids):
    arr = c_tensor.detach().cpu().numpy().flatten()
    idx = np.argmin(np.abs(arr[:, None] - centroids[None, :]), axis=1)
    return idx

def dequantize_c_lm(idx, centroids, shape):
    vals = centroids[idx]
    return torch.tensor(vals.reshape(shape), dtype=torch.float32, device=device)

In [ ]:
def encode_soft_prompt_lm(
    soft_prompt: torch.Tensor,
    prior_cdf_int: np.ndarray,
    centroids: np.ndarray,
    total_prior: int = 1 << 30
) -> list[int]:
    arr = soft_prompt.detach().cpu().numpy().flatten()
    c_idx = np.argmin(np.abs(arr[:, None] - centroids[None, :]), axis=1)

    enc = ArithmeticEncoder()
    for s in c_idx:
        lo, hi = prior_cdf_int[s], prior_cdf_int[s+1]
        enc.update(lo, hi, total_prior)
    enc.finish()

    return enc.out

In [ ]:
def decode_soft_prompt_lm(
    c_bits: list[int],
    prior_cdf_int: np.ndarray,
    centroids: np.ndarray,
    shape: tuple[int, ...],
    total_prior: int = 1 << 30,
    device: torch.device = torch.device('cpu')
) -> torch.Tensor:
    # 1) запустим арифметический декодер
    dec = ArithmeticDecoder(c_bits)

    # 2) сколько всего координат нужно восстановить
    n_elem = int(np.prod(shape))

    # 3) декодируем последовательность индексов
    rec_idx = [dec.decode(prior_cdf_int, total_prior) for _ in range(n_elem)]
    rec_idx = np.array(rec_idx, dtype=np.int64)

    # 4) деквантование: заменяем каждый индекс на значение центроида
    vals = centroids[rec_idx]           # shape (n_elem,)
    vals = vals.reshape(shape)          # восстанавливаем форму
    return torch.tensor(vals, device=device, dtype=torch.float32)

In [ ]:
prior_cdf_int, centroids, prior_pmf = lloyd_max_hyper_prior(soft_prompt)

In [ ]:
c_bits = encode_soft_prompt_lm(soft_prompt, prior_cdf_int, centroids)

In [ ]:
soft_prompt_rec = decode_soft_prompt_lm(
    c_bits,
    prior_cdf_int,
    centroids,
    tuple(soft_prompt.shape)
)

In [ ]:
evaluate_steering_coding(soft_prompt, soft_prompt_rec, prior_pmf, c_bits)

Число координат steering‑вектора: 102400
Битов использовано для кодирования: 788823
Битов на координату: 7.7033
Теоретическая энтропия на координату: 7.7030 бит, всего ≈788789.24 бит
MSE: 4.142708e-05
Коэффициент компрессии steering‑вектора: 0.2407


In [ ]:
soft_prompt_rec = soft_prompt_rec.to(device)

In [ ]:
chunk_data = encode_with_soft_prompt(
    dataset, model, soft_prompt_rec)

Encoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Encoding done in 202.83s


In [ ]:
all_rec = decode_with_soft_prompt(
    chunk_data, model, soft_prompt_rec)

Decoding chunks:   0%|          | 0/25 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/668 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/732 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/572 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/589 [00:00<?, ?it/s]

...

  Tokens in chunk:   0%|          | 0/494 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/495 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/442 [00:00<?, ?it/s]

  Tokens in chunk:   0%|          | 0/452 [00:00<?, ?it/s]

Decoding done in 202.14s


In [ ]:
orig_ids = sum((entry['ids'].tolist() for entry in chunk_data), [])
encoded_output = sum([entry['bits'] for entry in chunk_data], [])

In [ ]:
compare_sequences(orig_ids, all_rec)

✅  Совпадают полностью!


In [ ]:
compression_stats(data, encoded_output)

Исходный размер:   400000 бит
Размер после сжатия: 54156 бит
Коэффициент сжатия: 0.1354
